In [2]:
import numpy as np
import pandas as pd

In [32]:
### Source1 : Interpreting long-term trends in bushmeat harvest in southeast Cameroon
### Source2 : Consequences of survey method for estimating hunters'  harvest rates
### Source3 : Incentives and social relationships of hunters and traders in a Liberian bushmeat system

Population = pd.DataFrame(np.array([[82,71,85,143,163,152,98,81,71],[12,18,29,23,38,36,18,27,18]]),
                          columns=["DP2003", "DP2009", "DP2016", "M2003", "M2009", "M2016", "Mi2003", "Mi2009", "Mi2016"],
                          index=["Total Pop", "Total hunter"])

probaBeHunting = 10 / 365 #Source1
probaBeHunting2 = 25 / 365 #Source2
probaBeHunting3 = 5 / 365 #Source3

Population.loc["HW1"] = probaBeHunting * Population.loc["Total hunter"]
Population.loc["HW2"] = probaBeHunting2 * Population.loc["Total hunter"]
Population.loc["HW3"] = probaBeHunting3 * Population.loc["Total hunter"]


Population.insert(0, "mean", np.array([np.average(row) for _,row in Population.iterrows()]))

ms = [Population["mean"][k] / (Population["mean"]["Total Pop"] - Population["mean"][k]) for k in ["HW1", "HW2", "HW3"]]
print(ms)

[0.006382978723404255, 0.01611170784103115, 0.0031813361611876985]


In [38]:
### Source : Consommation alimentaire dans trois populations forestières de la région côtière du Cameroun: Yassa, Mvae et Bakola, page 490

muD = 1 / 60
m = 0.1

ConsoChasse = np.array([88, 70, 78])
ConsoLocal = np.array([10, 27, 18])
ConsoImportation = np.array([2,3,2])

Prop =  ConsoLocal / (ConsoChasse + ConsoImportation)
print(Prop)

fDs = Prop / (Prop + m) * muD
print("With proteins : ", fDs)
print("Average : ", np.average(fDs))
print(muD)

ConsoChasse = np.array([13, 12, 14])
ConsoLocal = np.array([82, 81, 81])
ConsoImportation = np.array([5,5,3])

Prop =  ConsoLocal / (ConsoChasse + ConsoImportation)
print(Prop)
fDs = Prop / (Prop + m) * muD
print("With calories : ", fDs)
print("Average : ", np.average(fDs))
print(muD)

[0.11111111 0.36986301 0.225     ]
With proteins :  [0.00877193 0.01311953 0.01153846]
Average :  0.011143308296906579
0.016666666666666666
[4.55555556 4.76470588 4.76470588]
With calories :  [0.01630867 0.01632406 0.01632406]
Average :  0.016318932398560002
0.016666666666666666


In [ ]:
### Source : DESA RAPPORT CFSAM 2019, région Est 2019

#### Culture ###
Culture = pd.DataFrame(np.array([[4840, 400], [18489, 19165], [30232, 20249], [840,105], [892657, 79175],[459043, 54343], [841323, 89106]]),
                       columns = ['Productions (T)', 'Superficies (Ha)'], index=["Igname", "Arachide", "Maïs", "Patate", "Banane Plantain", "Macabo", "Manioc"])

### Elevage ###
Elevage = pd.DataFrame(np.array([[6747],[492],[1206],[623],[2531]]),
                       columns = ['Prooductions(T)'], index=["Bovin", "Caprin", "Ovin", "Porcin", "Volailles"])

### Population ###
Pop = 771755



In [2]:
### Source : S. Gwangwa 'a et al. Habitudes alimentaires el état nutritionnel de la population d'Awing
# Bien manger et bien vivre, page 323

activites_productives = [14.3, 27.8, 29, 31.6] #heures par semaines
activites_productives = sum(activites_productives) / len(activites_productives)

print("Activites productives moyenne, en heure par semaine", activites_productives)
prop_activites_productives = activites_productives / (7 * 24)


Activites productives moyenne, en heure par semaine 25.674999999999997


In [6]:
### Source : Intrinsic rate of natural increase in Neotropical forest mammals: relationship to phylogeny and diet

Edentates = [0.39,0.69,0.79,0.59,0.43,0.48,0.35,0.26,0.14,0.23]
Primates = [0.23,0.24,0.25,0.17,0.14,0.16,0.11,0.15,0.17,0.07,0.10,0.07,0.14]
Lagomorphs = [11.51]
Rodents = [0.67,1.10,0.69]
Perissodactyls = [0.20]
Artiodactyls = [0.84,1.25,0.4,0.49,0.73]
Carnivores = [0.23,0.28,0.3,0.66,0.3,0.46,0.23]

All = np.array(Edentates + Primates + Lagomorphs + Rodents + Perissodactyls + Artiodactyls + Carnivores)

## At least Lagomorphs does not exist in Africa
NotAll = np.array(Edentates + Primates +  Rodents + Perissodactyls + Artiodactyls + Carnivores)
df = pd.DataFrame(NotAll)
print(df.describe())


               0
count  39.000000
mean    0.389231
std     0.285955
min     0.070000
25%     0.170000
50%     0.280000
75%     0.540000
max     1.250000


In [8]:
## Source : Ecological Structure of the Nonfiying Mammal Community at Cocha Cashu Biological Station, Manu National Park, Peru

density = [12,10,15,10,25,55,40,24,30,40,35,60,25,1,5,16,12,25,180,3.5,5.2,5.3,230,25.3,6.8,0.024,0.8,0.035,0.5,3,5.6,2.6]
totalPopulation = sum(density)

print(5*totalPopulation)

4543.295
